In [93]:
import tensorflow as tf

In [94]:
a = tf.constant(2)
b = tf.constant(3)
print(a)
sess = tf.Session()
l = tf.add(a, b)
print(l)
print(sess.run(l))
m = sess.run(a * b)
s = sess.run(a + b)
print(m, s)
sess.run([a * b, a + b])
a

Tensor("Const_50:0", shape=(), dtype=int32)
Tensor("Add_22:0", shape=(), dtype=int32)
5
6 5


<tf.Tensor 'Const_50:0' shape=() dtype=int32>

In [95]:
x = tf.placeholder(tf.int16)
y = tf.placeholder(tf.int16)
add = tf.add(x, y)
sess = tf.Session()
sess.run(add, feed_dict={x:10, y:20})

30

In [96]:
matrix1 = tf.constant([[3,3]])
matrix2 = tf.constant([[3],[3]])
product = tf.matmul(matrix1, matrix2)
with tf.Session() as sess:
    p = sess.run(product)
    print(p)
    

[[18]]


In [97]:
dummyinput = tf.constant([[1, 2], [3, 4]])
dummyweights = tf.constant([[4, 5, 6],[7, 8, 9]])
dummybiases = tf.constant([10,11,12])
product = tf.matmul(dummyinput, dummyweights)
sum1 = tf.add(product, dummybiases)
with tf.Session() as sess:
    p = sess.run(product)
    print(p)
    s = sess.run(sum1)
    print(s)

[[18 21 24]
 [40 47 54]]
[[28 32 36]
 [50 58 66]]


In [98]:
from tensorflow.examples.tutorials.mnist import input_data

In [99]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [100]:
mnist.train.images.shape
mnist.train.labels.shape

(55000, 10)

In [122]:
n_hidden_1 = 256
n_hidden_2 = 256
n_input = 784
n_classes = 10

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), trainable=False)
}

biases = {
    'h1': tf.Variable(tf.random_normal([n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


In [123]:
def mlp(x, weights, biases):
    layer1 = tf.add(tf.matmul(x, weights['h1']), biases['h1'])
    layer1 = tf.nn.relu(layer1)
    
    layer2 = tf.add(tf.matmul(layer1, weights['h2']), biases['h2'])
    layer2 = tf.nn.relu(layer2)
    
    out_layer = tf.add(tf.matmul(layer2, weights['out']), biases['out'])
    return out_layer

In [124]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
pred = mlp(x, weights, biases)
pred

<tf.Tensor 'Add_30:0' shape=(?, 10) dtype=float32>

In [125]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimize = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [136]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
sess.run(weights["out"])

array([[-0.14305195, -1.36674094, -0.43493849, ..., -1.80883133,
        -0.3080914 ,  0.07313408],
       [ 0.77560836,  0.01963067, -1.33096468, ..., -1.19808269,
        -1.25309873, -0.34445322],
       [-0.12336285,  0.7626853 , -1.22888339, ..., -1.28816831,
         0.41274747, -0.27915511],
       ..., 
       [-0.32510912, -0.91568601, -0.46606162, ...,  1.20296955,
         0.51508927,  0.47158167],
       [ 1.02626431,  0.72868288, -1.56767404, ..., -0.3387078 ,
         1.80937517,  1.1109215 ],
       [ 0.80470341, -0.03381886,  1.8886019 , ...,  1.69609678,
         0.26559997, -1.10651839]], dtype=float32)

In [131]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _ , c = sess.run([optimize, cost], feed_dict={x:batch_x, y:batch_y})
        total_cost += c
    print(total_cost)

24587.5243435
5408.00235936
3213.42695972
2261.7906509
2048.71928626
1826.3407236
1607.43707831
1601.06144383
1271.78305398
1493.92286433
1346.77477298
1201.60094269
1163.92642067
1121.93425646
897.90902987
939.053763835
1038.84749995
945.850463958
674.488912277
817.089539074
762.022508297
727.065790967
797.342816015
663.299333436
640.712804305


In [132]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_predictions  = sess.run([predictions, correct_predictions], feed_dict={x:mnist.test.images,
                                              y:mnist.test.labels})


In [134]:
(correct_predictions == False).sum()

291

In [135]:
sess.run(weights["out"])

array([[-0.63745224,  0.54393679, -1.01817393, ...,  0.13962531,
        -0.19499145, -0.58553326],
       [-1.07777834,  0.22162627, -1.86492717, ...,  0.04318257,
        -0.60116941, -0.06390394],
       [-1.26733768, -0.38340598, -0.27273902, ...,  1.18660736,
         0.13429713,  0.29988909],
       ..., 
       [ 2.0600841 ,  0.7561661 ,  1.03323066, ..., -0.76531082,
        -0.77663636,  0.45948863],
       [-0.58331132,  1.00319839, -0.02230331, ..., -0.79730892,
        -0.10656445, -0.19078746],
       [ 0.19489141, -0.74863994,  0.12361734, ..., -1.6110549 ,
        -2.67997646, -0.26950741]], dtype=float32)